# 실전 문제 해결 (과적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [4]:
class ConvLNReluBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvBNReluBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
        self.ln = tf.keras.layers.LayerNormalization()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        x = self.ln(x)
        return tf.nn.relu(x)
    

# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu',
                                           kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax',
                                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비


In [5]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

170500096/170498071 [==============================] - 31s 0us/step


## Keras API 모델 학습

In [ ]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 1563 steps, validate for 313 steps
Epoch 1/100
1563/1563 [==============================] - 218s 139ms/step - loss: 1.8829 - accuracy: 0.3650 - val_loss: 1.4717 - val_accuracy: 0.4936
Epoch 2/100
1563/1563 [==============================] - 206s 132ms/step - loss: 1.3639 - accuracy: 0.5408 - val_loss: 1.2503 - val_accuracy: 0.5865
Epoch 3/100
1563/1563 [==============================] - 207s 132ms/step - loss: 1.1908 - accuracy: 0.6109 - val_loss: 1.2133 - val_accuracy: 0.6011
Epoch 4/100
1494/1563 [===========================>..] - ETA: 9s - loss: 1.0931 - accuracy: 0.6460